In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Make plots look nicer
sns.set_theme(style="whitegrid", context="talk")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["figure.dpi"] = 120

In [ ]:
df = pd.read_csv("Access_to_Everyday_Life_Dataset.csv")

In [ ]:
df = df.rename(columns={
    "geometry/coordinates/0": "lon",
    "geometry/coordinates/1": "lat",
    "properties/attribute_id": "attribute_id",
    "properties/label_type": "label_type",
    "properties/neighborhood": "neighborhood",
    "properties/severity": "severity",
    "properties/is_temporary": "is_temporary",
    "geometry/type": "geometry_type",
    "type": "feature_type"
})

df.head()


In [ ]:
df["severity"] = pd.to_numeric(df["severity"], errors="coerce")

# Sometimes these come as TRUE/FALSE strings
if df["is_temporary"].dtype == "object":
    df["is_temporary"] = df["is_temporary"].astype(str).str.upper().map({"TRUE": True, "FALSE": False})

df[["severity", "is_temporary"]].dtypes


In [ ]:
(df.isna().mean().sort_values(ascending=False) * 100).round(2)


In [ ]:
import pandas as pd
import numpy as np
import folium

from folium.plugins import (
    HeatMap, MarkerCluster, MiniMap, Fullscreen, LocateControl,
    MeasureControl, Draw, Geocoder
)
from folium import IFrame

# -------------------------
# 0) Setup + safety cleaning
# -------------------------
needed = ["lat", "lon", "severity", "label_type", "neighborhood", "is_temporary"]
for c in needed:
    if c not in df.columns:
        raise KeyError(f"Missing column '{c}'. Current columns: {list(df.columns)}")

# Ensure correct types (optional but helps)
df = df.copy()
df["severity"] = pd.to_numeric(df["severity"], errors="coerce")
if df["is_temporary"].dtype == "object":
    df["is_temporary"] = df["is_temporary"].astype(str).str.upper().map({"TRUE": True, "FALSE": False})

center = [df["lat"].mean(), df["lon"].mean()]

# -------------------------
# 1) Base map
# -------------------------
m = folium.Map(location=center, zoom_start=12, tiles="CartoDB positron")

# Global “inspect mode” tools on the whole map
MiniMap(toggle_display=True).add_to(m)
Fullscreen(position="topright").add_to(m)
LocateControl(auto_start=False).add_to(m)
MeasureControl(position="topleft").add_to(m)

# Search / geocoder
Geocoder(collapsed=True, add_marker=True).add_to(m)

# Draw tool (exports selection)
Draw(
    export=True,
    filename="selected_area.geojson",
    position="topleft",
    draw_options={"polyline": False, "circle": False, "circlemarker": False}
).add_to(m)

# ---------------------------------------------------
# 2) View A: Interactive Heatmap (severity-weighted)
# ---------------------------------------------------
heat_fg = folium.FeatureGroup(name="VIEW: Heatmap (weighted by severity)", show=False)
heat_data = df[["lat", "lon", "severity"]].dropna().values.tolist()
HeatMap(heat_data, radius=10, blur=15, max_zoom=13).add_to(heat_fg)
heat_fg.add_to(m)

# ----------------------------------------------------------------
# 3) View B: Clustered Markers (simple CircleMarker + popup)
# ----------------------------------------------------------------
cluster_fg = folium.FeatureGroup(name="VIEW: Clustered Markers (basic)", show=False)
cluster = MarkerCluster().add_to(cluster_fg)

sample_basic = df.dropna(subset=["lat","lon"]).sample(min(3000, len(df)), random_state=7)

for _, r in sample_basic.iterrows():
    popup = (f"Type: {r['label_type']}<br>"
             f"Neighborhood: {r['neighborhood']}<br>"
             f"Severity: {r['severity']}<br>"
             f"Temporary: {r['is_temporary']}")
    folium.CircleMarker(
        location=[r["lat"], r["lon"]],
        radius=3,
        popup=popup,
        fill=True
    ).add_to(cluster)

cluster_fg.add_to(m)

# ---------------------------------------------------------------------
# 4) View C: Clustered Markers (rich HTML popup w/ severity badge)
# ---------------------------------------------------------------------
badge_fg = folium.FeatureGroup(name="VIEW: Clustered Markers (rich popups)", show=False)
badge_cluster = MarkerCluster().add_to(badge_fg)

sample_badge = df.dropna(subset=["lat","lon","severity"]).sample(min(2000, len(df)), random_state=7)

for _, r in sample_badge.iterrows():
    sev = int(r["severity"]) if pd.notna(r["severity"]) else "NA"
    temp = r["is_temporary"]

    html = f"""
    <div style="font-family: Arial; font-size: 14px;">
      <div style="font-size:16px; font-weight:700;">{r['label_type']}</div>
      <div><b>Neighborhood:</b> {r['neighborhood']}</div>
      <div><b>Severity:</b>
        <span style="padding:2px 6px; border-radius:8px; border:1px solid #999;">
          {sev}
        </span>
      </div>
      <div><b>Temporary:</b> {temp}</div>
      <div style="margin-top:6px; color:#666;">
        ({r['lat']:.5f}, {r['lon']:.5f})
      </div>
    </div>
    """
    iframe = IFrame(html=html, width=270, height=150)
    folium.Marker([r["lat"], r["lon"]], popup=folium.Popup(iframe)).add_to(badge_cluster)

badge_fg.add_to(m)

# -------------------------------------------------------------
# 5) View D: Toggles (Permanent vs Temporary vs Severe ≥ 4)
# -------------------------------------------------------------
toggle_group = folium.FeatureGroup(name="VIEW: Toggle layers (perm/temp/severe)", show=False)

fg_perm = folium.FeatureGroup(name="Permanent (not temporary)")
fg_temp = folium.FeatureGroup(name="Temporary")
fg_severe = folium.FeatureGroup(name="Severe (severity ≥ 4)")

clean = df.dropna(subset=["lat","lon","severity","is_temporary"]).sample(min(5000, len(df)), random_state=7)

for _, r in clean.iterrows():
    popup = (f"Type: {r['label_type']}<br>"
             f"Neighborhood: {r['neighborhood']}<br>"
             f"Severity: {r['severity']}<br>"
             f"Temporary: {r['is_temporary']}")

    marker = folium.CircleMarker(
        location=[r["lat"], r["lon"]],
        radius=3,
        popup=popup,
        fill=True
    )

    if r["is_temporary"] == True:
        marker.add_to(fg_temp)
    else:
        marker.add_to(fg_perm)

    if r["severity"] >= 4:
        folium.CircleMarker(
            location=[r["lat"], r["lon"]],
            radius=4,
            popup=popup,
            fill=True
        ).add_to(fg_severe)

fg_perm.add_to(toggle_group)
fg_temp.add_to(toggle_group)
fg_severe.add_to(toggle_group)

toggle_group.add_to(m)

# -------------------------------------------------------------
# 6) View E: Choropleth-like severity grid overlay (rectangles)
# -------------------------------------------------------------
def make_grid(df_in, cell_size=0.003):
    d = df_in.dropna(subset=["lat","lon","severity"]).copy()
    d["gx"] = (d["lon"] / cell_size).astype(int)
    d["gy"] = (d["lat"] / cell_size).astype(int)

    agg = d.groupby(["gx","gy"]).agg(
        n=("severity","size"),
        avg_sev=("severity","mean")
    ).reset_index()

    agg["lon_min"] = agg["gx"] * cell_size
    agg["lon_max"] = (agg["gx"] + 1) * cell_size
    agg["lat_min"] = agg["gy"] * cell_size
    agg["lat_max"] = (agg["gy"] + 1) * cell_size
    return agg

grid_fg = folium.FeatureGroup(name="VIEW: Severity grid overlay", show=False)
grid = make_grid(df, cell_size=0.003)

for _, r in grid.iterrows():
    if r["n"] < 5:
        continue

    opacity = min(0.85, max(0.1, (r["avg_sev"] - 1) / 4))

    folium.Rectangle(
        bounds=[[r["lat_min"], r["lon_min"]], [r["lat_max"], r["lon_max"]]],
        fill=True,
        fill_opacity=opacity,
        weight=0,
        popup=f"Count: {int(r['n'])}<br>Avg severity: {r['avg_sev']:.2f}"
    ).add_to(grid_fg)

grid_fg.add_to(m)

# -------------------------------------------------------------
# 7) View F: Separate layers by top label_type (toggle within view)
# -------------------------------------------------------------
types_view = folium.FeatureGroup(name="VIEW: Barrier types (top 6 toggles)", show=False)

clean2 = df.dropna(subset=["lat","lon","label_type","severity"])
top_types = clean2["label_type"].value_counts().head(6).index

type_layers = {t: folium.FeatureGroup(name=f"Type: {t}") for t in top_types}

sample_types = clean2[clean2["label_type"].isin(top_types)].sample(min(6000, len(clean2)), random_state=7)
for _, r in sample_types.iterrows():
    popup = (f"Type: {r['label_type']}<br>"
             f"Neighborhood: {r['neighborhood']}<br>"
             f"Severity: {r['severity']}")
    folium.CircleMarker(
        location=[r["lat"], r["lon"]],
        radius=3,
        popup=popup,
        fill=True
    ).add_to(type_layers[r["label_type"]])

for t, fg in type_layers.items():
    fg.add_to(types_view)

types_view.add_to(m)

# -------------------------
# 8) Layer control + save
# -------------------------
# "collapsed=False" keeps the panel open (feels like buttons)
folium.LayerControl(collapsed=False).add_to(m)

m.save("all_views_accessibility_map.html")
m
